In [1]:
import json
import re
from os import listdir
import skimage.io as io
from shapely.geometry import Polygon, shape, Point
from osgeo import gdal, osr, ogr, gdalnumeric
import numpy as np
import sys
import os
import matplotlib.pyplot as plt
from matplotlib.collections import PatchCollection
from matplotlib.patches import Polygon
from shapely.geometry import Polygon
import pylab
from area import area
from datetime import datetime
%matplotlib inline 

In [2]:
def latlon2pixel(lat, lon, input_raster='', targetsr='', geom_transform=''):
    # type: (object, object, object, object, object) -> object

    sourcesr = osr.SpatialReference()
    sourcesr.ImportFromEPSG(4326)

    geom = ogr.Geometry(ogr.wkbPoint)
    geom.AddPoint(lon, lat)

    if targetsr == '':
        src_raster = gdal.Open(input_raster)
        targetsr = osr.SpatialReference()
        targetsr.ImportFromWkt(src_raster.GetProjectionRef())
    coord_trans = osr.CoordinateTransformation(sourcesr, targetsr)
    if geom_transform == '':
        src_raster = gdal.Open(input_raster)
        transform = src_raster.GetGeoTransform()
    else:
        transform = geom_transform

    x_origin = transform[0]
    # print(x_origin)
    y_origin = transform[3]
    # print(y_origin)
    pixel_width = transform[1]
    # print(pixel_width)
    pixel_height = transform[5]
    # print(pixel_height)
    geom.Transform(coord_trans)
    # print(geom.GetPoint())
    x_pix = (geom.GetPoint()[0] - x_origin) / pixel_width
    y_pix = (geom.GetPoint()[1] - y_origin) / pixel_height

    return (x_pix, y_pix)

def geoPolygonToPixelPolygonWKT(geom, inputRaster, targetSR, geomTransform, breakMultiPolygonGeo=True,
                                pixPrecision=2):
    # Returns Pixel Coordinate List and GeoCoordinateList
    return_array = []
    polygonPixBufferList = []
    polygonPixBufferWKTList = []
    polygonGeoWKTList = []
    if geom.GetGeometryName() == 'POLYGON':
        polygonPix = ogr.Geometry(ogr.wkbPolygon)
        for ring in geom:
            # GetPoint returns a tuple not a Geometry
            ringPix = ogr.Geometry(ogr.wkbLinearRing)

            for pIdx in range(ring.GetPointCount()):
                lon, lat, z = ring.GetPoint(pIdx)
                xPix, yPix = latlon2pixel(lat, lon, inputRaster, targetSR, geomTransform)

                xPix = round(xPix, pixPrecision)
                yPix = round(yPix, pixPrecision)
                ringPix.AddPoint(xPix, yPix)
                
                return_array.append(xPix)
                return_array.append(yPix)

            polygonPix.AddGeometry(ringPix)
        polygonPixBuffer = polygonPix.Buffer(0.0)
        polygonPixBufferList.append([polygonPixBuffer, geom])

    elif geom.GetGeometryName() == 'MULTIPOLYGON':

        for poly in geom:
            polygonPix = ogr.Geometry(ogr.wkbPolygon)
            for ring in poly:
                # GetPoint returns a tuple not a Geometry
                ringPix = ogr.Geometry(ogr.wkbLinearRing)

                for pIdx in range(ring.GetPointCount()):
                    lon, lat, z = ring.GetPoint(pIdx)
                    xPix, yPix = latlon2pixel(lat, lon, inputRaster, targetSR, geomTransform)

                    xPix = round(xPix, pixPrecision)
                    yPix = round(yPix, pixPrecision)
                    ringPix.AddPoint(xPix, yPix)
                    
                    return_array.append(xPix)
                    return_array.append(yPix)

                polygonPix.AddGeometry(ringPix)
            polygonPixBuffer = polygonPix.Buffer(0.0)
            if breakMultiPolygonGeo:
                polygonPixBufferList.append([polygonPixBuffer, poly])
            else:
                polygonPixBufferList.append([polygonPixBuffer, geom])

    for polygonTest in polygonPixBufferList:
        if polygonTest[0].GetGeometryName() == 'POLYGON':
            polygonPixBufferWKTList.append([polygonTest[0].ExportToWkt(), polygonTest[1].ExportToWkt()])
        elif polygonTest[0].GetGeometryName() == 'MULTIPOLYGON':
            for polygonTest2 in polygonTest[0]:
                polygonPixBufferWKTList.append([polygonTest2.ExportToWkt(), polygonTest[1].ExportToWkt()])

    return return_array, polygonPixBufferWKTList

In [3]:
#initialize data for output json
my_categories = [{"id": 1,"name": "building", "supercategory": "structure"}]
my_info = {"description": "This is stable 1.0 version of the Vegas SpaceNet dataset in COCO format.", "url": "none", "version": "1.0", "year": 2017, "contributor": "Lee Cohn", "date_created": "2017-04-17 00:00:00.000000"}
my_images = []
my_licenses = [{"id": 1,"name": "Vegas-SpaceNet","url": "none"}]
my_annotations = []

In [7]:
def process_directory(feature_index):
    #Loop through directory of tif files
    for rasterSrc_file in listdir('vegas_tif'):
        
        #open raster and vector
        srcRas_ds = gdal.Open('vegas_tif/'+rasterSrc_file)
        print("Raster file : ",'vegas_tif/'+rasterSrc_file)
        cols = srcRas_ds.RasterXSize
        rows = srcRas_ds.RasterYSize
        

        #Get image number
        image_number_search = re.search('(?<=img)\w+', rasterSrc_file)
        image_number = image_number_search.group(0)
        image_number = int(image_number)
        
        jpg_file = rasterSrc_file.replace('.tif', '.jpg')
        
        
        #create image json entry
        image_json_entry = {"license": 1, "file_name": jpg_file, "coco_url": "none", "height": rows, "width": cols, "date_captured": str(datetime.now()), "flickr_url": "none", "id": image_number}
        my_images.append(image_json_entry)

        vectorSrc_file = 'buildings_AOI_2_Vegas_img'+str(image_number)+'.geojson'
       
        source_ds = ogr.Open('vegas_geojson/'+vectorSrc_file)
        source_layer = source_ds.GetLayer()
        num_features = source_layer.GetFeatureCount()
        
        
        for feature in source_layer:
            my_geom = feature.GetGeometryRef()
            if(my_geom.GetGeometryName() == 'POINT'):
                feature_index = feature_index + 1
                continue
                
            test = geoPolygonToPixelPolygonWKT(my_geom, 'vegas_tif/'+rasterSrc_file, '', '', breakMultiPolygonGeo=True, pixPrecision=2)
            my_list = test[0]
            poly = np.array(my_list).reshape((int(len(my_list)/2), 2))
            my_poly = Polygon(poly)
            x_min, y_min, x_max, y_max = my_poly.bounds
            width = x_max - x_min
            height = y_max - y_min
            my_bbox = [x_min,y_min,width,height]
            
            #create feature json entry
            feature_json_entry = {"id": feature_index,"image_id": image_number,"category_id": 1,"segmentation": [my_list],"area": my_poly.area,"bbox": my_bbox,"iscrowd": 0}
            my_annotations.append(feature_json_entry)
          
            feature_index = feature_index + 1

feature_index = 1            
process_directory(feature_index)
print("DONE")

('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img5624.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img6353.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img5500.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img2555.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img3707.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img4347.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img1070.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img287.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img6080.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img2915.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img838.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img2918.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img3038.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img3943.tif')
('Raster

('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img4098.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img1139.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img3002.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img706.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img3338.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img781.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img3825.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img5182.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img6018.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img1584.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img637.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img6305.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img5205.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img6265.tif')
('Raster 

('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img2524.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img5061.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img1418.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img1604.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img2856.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img827.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img784.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img5381.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img1952.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img3029.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img1532.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img531.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img1514.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img3851.tif')
('Raster 

('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img3922.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img2538.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img3807.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img3117.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img410.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img5235.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img5599.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img5678.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img4314.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img488.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img5429.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img2367.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img1649.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img2476.tif')
('Raster

('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img749.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img3731.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img4486.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img1.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img2942.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img5230.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img1505.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img4010.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img1827.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img6189.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img4058.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img4319.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img174.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img3857.tif')
('Raster fi

...

('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img4253.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img216.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img4364.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img1293.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img3651.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img1783.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img3288.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img4999.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img1489.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img1620.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img5973.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img3304.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img1802.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img1500.tif')
('Raste

('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img3242.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img5648.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img4725.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img1960.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img2079.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img6032.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img3392.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img888.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img1267.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img6132.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img3520.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img3078.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img190.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img1451.tif')
('Raster

('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img3888.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img3330.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img4304.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img4282.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img4093.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img2219.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img5859.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img5423.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img6276.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img3384.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img2061.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img6293.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img3060.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img1308.tif')
('Rast

('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img3646.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img2569.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img37.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img3269.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img1215.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img5107.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img2578.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img4505.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img2724.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img5714.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img1612.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img4608.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img2095.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img971.tif')
('Raster 

('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img3068.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img4810.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img1227.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img483.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img1749.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img5359.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img3733.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img6035.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img2741.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img6159.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img2393.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img6193.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img165.tif')
('Raster file : ', 'vegas_tif/RGB-PanSharpen_AOI_2_Vegas_img4648.tif')
('Raster

In [ ]:
json_data = {"info":my_info, "images":my_images, "annotations":my_annotations, "licenses":my_licenses, "categories":my_categories}

with open("vegas_train_instances_2.json", "w") as outfile:
    json.dump(json_data, outfile, indent=4)